In [1]:
import pandas as pd 
from pandas import DataFrame, Series
import os, sys, re
import numpy as np

from decimal import Decimal
from tabulate import tabulate

import warnings
warnings.filterwarnings('ignore')  # action='once'

%matplotlib inline

## Utility Functions

In [2]:
from analyzer import interpret

def col_values(df, col='age', n=10):
    df_subset = df.sample(n=n, random_state=1) # for each column, sample n values (usually n=1)
    return df_subset[col].values

def summary(df, n=1): 
    msg = ""
    msg += "> sample sizes: {}\n".format(df.shape[0])
    msg += "> n(features):  {}\n".format(df.shape[1])
    # msg += "> list of features:\n{}\n".format(df.columns.values)
    print(msg)

    interpret(df, n=n, verbose=True)
def show_dict(adict, topn=-1, by='', header=[], n_samples=-1, ascending=False, print_=False): 
    # print(adict)
    
    # convert to two-column dataframe format 
    if not header: 
        header = ['key', 'value']
    else: 
        assert len(header) == 2
    D = {h:[] for h in header}
    for k, v in adict.items(): 
        D[header[0]].append(k)
        D[header[1]].append(v)
    
    df = DataFrame(D, columns=header)
    msg = ''
    if topn > 0: 
        assert by in df.columns
        df = df.sort_values([by, ], ascending=ascending)
        msg = tabulate(df[:topn], headers='keys', tablefmt='psql')
    else: 
        if n_samples < 0: 
            msg = tabulate(df, headers='keys', tablefmt='psql')
        else: 
            n = min(df.shape[0], n_samples)
            msg = tabulate(df.sample(n=n), headers='keys', tablefmt='psql')
    if print_: print(msg)
    return msg

### Determine and Retrieve Patient Cohort(s)

In [3]:
"""
1. Find rows whose column match a substring 
   https://davidhamann.de/2017/06/26/pandas-select-elements-by-string/
   
   https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html

"""
import json 
from cohort_search import gen_code_set, gen_query_str

# use module cohort_search to generate the desired query strings 
# run CohortRetrieval on Databricks (https://dbc-25924283-13f6.cloud.databricks.com/#notebook/1971793/command/1973735)

### Load Base/Positive Dataset

In [4]:
from analyzer import load_src_data, load_data, save_data, load_performance, stratify
# from transformer import canonicalize  # ... obsolete
from transformer import resolve_duplicate
import loinc as lc
# from loinc import canonicalize

######################################
# params
cohort = domain = 'hepatitis-c'
token_default = 'unknown'
col_target = 'test_result_loinc_code'
######################################

# load source data
ts0 = load_src_data(cohort=cohort, warn_bad_lines=False, canonicalized=True, processed=False)

print("> dim(ts0): {}".format(ts0.shape))  
loinc_set = codes0 = ts0[col_target].unique()
codes0_subset = np.random.choice(codes0, 30)
print("> N(loinc_set): {} | example codes (source):\n{}\n".format(len(loinc_set), list(codes0_subset)))

# stratify the data by class labels (e.g. LOINC codes)
ds = stratify(ts0, col=col_target) # code|mean|std|n_pos
Nmax0 = ds[0][1]  
# ... here we have not added extra control data yet (i.e. patients without the target disease such hepatitis C)

# compare to the previously generated performance dataframe
df_perf = load_performance(input_dir='result', cohort=cohort)
df_perf = df_perf.sort_values(by=['n_pos', ], ascending=False)
Nmax = np.max(df_perf['n_pos'].values)
Nm = np.median(df_perf['n_pos'].values)

ss_dict = dict(zip(df_perf['code'].values, df_perf['n_pos'].values))
print("> Set the baseline class sample size: Nmax: {} (Nmax0: {}, median: {})".format(Nmax, Nmax0, Nm))

topn = 10
print("> top {} sample sizes:\n{}\n".format(topn, show_dict(ss_dict, topn=topn, 
        by='n_pos', header=['code', 'n_pos'])))

######################################
"""
Conclusion 
----------

Cohort: hepatitis-c

1. max sample size with known code: 67686 => 57202
2. loinc_set: target loinc codes
   size(loinc_set): 733
3. ts0: dim=(71224, 127)


"""

[load] Loading default input data: andromeda-pond-hepatitis-c.csv
(canonicalize) Operations: fill n/a + dehyphenate + replace_values + trim_tail + fill others (non-target classes)
> dim(ts0): (71224, 127)
> N(loinc_set): 733 | example codes (source):
['64717', '21626', '422410', '143388', '265249', '163485', '268748', '178566', '139501', '265116', '264747', '303644', '433045', '264648', '24588', '93187', '109009', '295980', '301804', '298935', '264663', '177915', '192997', '17426', '193433', '169490', '320184', '64634', '339143', '502211']

> dim(performance matrix): (733, 5)
> Set the baseline class sample size: Nmax: 5593 (Nmax0: 5593, median: 3.0)
> top 10 sample sizes:
+----+---------+---------+
|    | code    |   n_pos |
|----+---------+---------|
|  0 | unknown |    5593 |
|  1 | 67686   |    3120 |
|  2 | 110114  |    2521 |
|  3 | 19752   |    2423 |
|  4 | 17426   |    2405 |
|  5 | 17517   |    2368 |
|  6 | 7187    |    2037 |
|  7 | 7773    |    2037 |
|  8 | 21600   |    2

'\nConclusion \n----------\n\nCohort: hepatitis-c\n\n1. max sample size with known code: 67686 => 57202\n2. loinc_set: target loinc codes\n   size(loinc_set): 733\n3. ts0: dim=(71224, 127)\n\n\n'

### Load Control Data

In [5]:
# patients not in given domain/disease
from cohort_search import filter_by_diagnosis
from analyzer import save_data
from transformer import resolve_duplicate

######################################
# params
tFilter = False
tSave = True
tAddCtrl = True
output_dir = 'data'
######################################

N0 = ts0.shape[0]

# load source data
ts_ctrl = load_src_data(input_file='andromeda_pond-10p.csv', canonicalized=True)
# ts_ctrl = ts_ctrl.drop_duplicates(keep='last')  # drop duplicates
# ts_ctrl = resolve_duplicate(ts_ctrl)
# ... generate a new variable: 'count'
# ts_ctrl = lc.canonicalize(ts_ctrl, col_target=col_target, token_missing=token_default, target_labels=loinc_set)
# ... now included by default within load_data()

print("> dim(ts_ctrl): {}".format(ts_ctrl.shape))  

if tFilter: 
    ts_ctrl = filter_by_diagnosis(ts_ctrl, condition='hepatitis c')
    nctrl = ts_ctrl.shape[0]
    print("> sample size | orig: {}, filtered(control): {}".format(n0, nctrl))

# summary(df_ctrl, n=1)
# df_ctrl.info()   

# save 
if tSave: 
    output_file = f"andromeda-pond-{cohort}-ctrl.csv" 
    save_data(ts_ctrl,  output_file=output_file, sep=',')
    
# mix-in 
if tAddCtrl: 
    Nctrl = ts_ctrl.shape[0]
    assert Nctrl > N0, f"Control data is too small | n({cohort})={N0} > n(ctrl)={Nctrl}"
    ts_ctrl = ts_ctrl.sample(n=N0, replace=False)
    
    ts0 = pd.concat([ts0, ts_ctrl], ignore_index=True)
    
assert np.sum(ts0[col_target].isnull()) == 0
print("> Adding control data | N: {} -> {}".format(N0, ts0.shape[0]))

[load] Loading default input data: andromeda_pond-10p.csv


b'Skipping line 256: expected 127 fields, saw 128\nSkipping line 511: expected 127 fields, saw 129\nSkipping line 5917: expected 127 fields, saw 133\nSkipping line 7086: expected 127 fields, saw 128\nSkipping line 7750: expected 127 fields, saw 128\n'
b'Skipping line 10545: expected 127 fields, saw 131\nSkipping line 14112: expected 127 fields, saw 128\nSkipping line 14113: expected 127 fields, saw 128\nSkipping line 15206: expected 127 fields, saw 135\n'
b'Skipping line 20277: expected 127 fields, saw 130\n'
b'Skipping line 25255: expected 127 fields, saw 136\nSkipping line 30914: expected 127 fields, saw 164\n'
b'Skipping line 34649: expected 127 fields, saw 130\nSkipping line 37064: expected 127 fields, saw 131\nSkipping line 39673: expected 127 fields, saw 128\nSkipping line 39674: expected 127 fields, saw 128\nSkipping line 39676: expected 127 fields, saw 128\nSkipping line 40260: expected 127 fields, saw 132\nSkipping line 40291: expected 127 fields, saw 129\nSkipping line 40614:

b'Skipping line 312711: expected 127 fields, saw 132\nSkipping line 313105: expected 127 fields, saw 128\nSkipping line 314345: expected 127 fields, saw 128\nSkipping line 315877: expected 127 fields, saw 129\nSkipping line 317175: expected 127 fields, saw 129\n'
b'Skipping line 323634: expected 127 fields, saw 130\nSkipping line 323650: expected 127 fields, saw 131\n'
b'Skipping line 331833: expected 127 fields, saw 148\nSkipping line 332177: expected 127 fields, saw 142\nSkipping line 334586: expected 127 fields, saw 130\n'
b'Skipping line 336654: expected 127 fields, saw 128\nSkipping line 341873: expected 127 fields, saw 128\n'
b'Skipping line 345036: expected 127 fields, saw 129\nSkipping line 346459: expected 127 fields, saw 131\nSkipping line 346788: expected 127 fields, saw 134\nSkipping line 348949: expected 127 fields, saw 130\nSkipping line 348994: expected 127 fields, saw 128\n'
b'Skipping line 353627: expected 127 fields, saw 130\nSkipping line 355166: expected 127 fields,

b'Skipping line 590629: expected 127 fields, saw 130\nSkipping line 595449: expected 127 fields, saw 130\nSkipping line 595463: expected 127 fields, saw 130\nSkipping line 596527: expected 127 fields, saw 130\n'
b'Skipping line 599747: expected 127 fields, saw 131\nSkipping line 599973: expected 127 fields, saw 141\nSkipping line 600752: expected 127 fields, saw 128\n'
b'Skipping line 606853: expected 127 fields, saw 129\nSkipping line 608275: expected 127 fields, saw 128\n'
b'Skipping line 615975: expected 127 fields, saw 129\nSkipping line 619136: expected 127 fields, saw 128\nSkipping line 620929: expected 127 fields, saw 130\n'
b'Skipping line 628790: expected 127 fields, saw 128\nSkipping line 628793: expected 127 fields, saw 128\nSkipping line 628794: expected 127 fields, saw 128\nSkipping line 629569: expected 127 fields, saw 128\n'
b'Skipping line 631936: expected 127 fields, saw 128\nSkipping line 632185: expected 127 fields, saw 128\nSkipping line 632331: expected 127 fields,

b'Skipping line 885532: expected 127 fields, saw 128\nSkipping line 888688: expected 127 fields, saw 130\nSkipping line 890608: expected 127 fields, saw 130\nSkipping line 891400: expected 127 fields, saw 128\n'
b'Skipping line 893962: expected 127 fields, saw 134\nSkipping line 894471: expected 127 fields, saw 128\nSkipping line 897155: expected 127 fields, saw 131\nSkipping line 897175: expected 127 fields, saw 129\n'
b'Skipping line 905330: expected 127 fields, saw 128\nSkipping line 907408: expected 127 fields, saw 128\nSkipping line 907851: expected 127 fields, saw 131\n'
b'Skipping line 910890: expected 127 fields, saw 132\nSkipping line 910933: expected 127 fields, saw 129\nSkipping line 911173: expected 127 fields, saw 135\nSkipping line 911180: expected 127 fields, saw 144\nSkipping line 913057: expected 127 fields, saw 129\nSkipping line 914740: expected 127 fields, saw 128\nSkipping line 917499: expected 127 fields, saw 128\nSkipping line 917500: expected 127 fields, saw 130

...

b'Skipping line 2008931: expected 127 fields, saw 130\nSkipping line 2010181: expected 127 fields, saw 128\nSkipping line 2011005: expected 127 fields, saw 129\nSkipping line 2011192: expected 127 fields, saw 134\nSkipping line 2012724: expected 127 fields, saw 132\nSkipping line 2013700: expected 127 fields, saw 128\nSkipping line 2014254: expected 127 fields, saw 130\nSkipping line 2014420: expected 127 fields, saw 130\n'
b'Skipping line 2016602: expected 127 fields, saw 128\nSkipping line 2019017: expected 127 fields, saw 128\nSkipping line 2019018: expected 127 fields, saw 128\nSkipping line 2019019: expected 127 fields, saw 128\nSkipping line 2019020: expected 127 fields, saw 128\nSkipping line 2020212: expected 127 fields, saw 128\nSkipping line 2020213: expected 127 fields, saw 128\nSkipping line 2020214: expected 127 fields, saw 128\nSkipping line 2020977: expected 127 fields, saw 146\n'
b'Skipping line 2026787: expected 127 fields, saw 128\nSkipping line 2032171: expected 127 

b'Skipping line 2280427: expected 127 fields, saw 128\nSkipping line 2280428: expected 127 fields, saw 128\nSkipping line 2280429: expected 127 fields, saw 128\nSkipping line 2281041: expected 127 fields, saw 130\nSkipping line 2282551: expected 127 fields, saw 128\nSkipping line 2282552: expected 127 fields, saw 128\nSkipping line 2282556: expected 127 fields, saw 128\nSkipping line 2282668: expected 127 fields, saw 128\nSkipping line 2282670: expected 127 fields, saw 128\nSkipping line 2282671: expected 127 fields, saw 128\nSkipping line 2284235: expected 127 fields, saw 132\nSkipping line 2284294: expected 127 fields, saw 128\nSkipping line 2284972: expected 127 fields, saw 130\nSkipping line 2285291: expected 127 fields, saw 132\n'
b'Skipping line 2287700: expected 127 fields, saw 129\nSkipping line 2290998: expected 127 fields, saw 128\nSkipping line 2291745: expected 127 fields, saw 128\nSkipping line 2292079: expected 127 fields, saw 144\n'
b'Skipping line 2296407: expected 127 

b'Skipping line 2551401: expected 127 fields, saw 130\nSkipping line 2552742: expected 127 fields, saw 131\nSkipping line 2556787: expected 127 fields, saw 128\nSkipping line 2556788: expected 127 fields, saw 128\nSkipping line 2556900: expected 127 fields, saw 134\n'
b'Skipping line 2559847: expected 127 fields, saw 128\nSkipping line 2559848: expected 127 fields, saw 128\nSkipping line 2559849: expected 127 fields, saw 128\nSkipping line 2559850: expected 127 fields, saw 128\nSkipping line 2559851: expected 127 fields, saw 128\nSkipping line 2561384: expected 127 fields, saw 128\nSkipping line 2561413: expected 127 fields, saw 130\nSkipping line 2565166: expected 127 fields, saw 130\nSkipping line 2565198: expected 127 fields, saw 129\n'
b'Skipping line 2566270: expected 127 fields, saw 132\nSkipping line 2567912: expected 127 fields, saw 130\nSkipping line 2568102: expected 127 fields, saw 144\nSkipping line 2569523: expected 127 fields, saw 130\n'
b'Skipping line 2574653: expected 

b'Skipping line 2820314: expected 127 fields, saw 131\nSkipping line 2823853: expected 127 fields, saw 128\nSkipping line 2824079: expected 127 fields, saw 130\n'
b'Skipping line 2831076: expected 127 fields, saw 129\nSkipping line 2832000: expected 127 fields, saw 128\n'
b'Skipping line 2836429: expected 127 fields, saw 130\nSkipping line 2836540: expected 127 fields, saw 135\nSkipping line 2838034: expected 127 fields, saw 128\nSkipping line 2842677: expected 127 fields, saw 128\nSkipping line 2842695: expected 127 fields, saw 131\nSkipping line 2843575: expected 127 fields, saw 130\n'
b'Skipping line 2845399: expected 127 fields, saw 128\nSkipping line 2845400: expected 127 fields, saw 128\nSkipping line 2845401: expected 127 fields, saw 128\nSkipping line 2845796: expected 127 fields, saw 128\nSkipping line 2847493: expected 127 fields, saw 130\nSkipping line 2848922: expected 127 fields, saw 128\nSkipping line 2849326: expected 127 fields, saw 128\n'
b'Skipping line 2852507: expec

(canonicalize) Operations: fill n/a + dehyphenate + replace_values + trim_tail + fill others (non-target classes)
> dim(ts_ctrl): (2891340, 127)
(save_data) Saved dataframe (dim=(2891340, 127)) to:
/Users/barnett/Documents/work/loinc_predictor/data/andromeda-pond-hepatitis-c-ctrl.csv

> Adding control data | N: 71224 -> 142448


### Define Feature Set 

Note: Subsetting columns could potentially reduce the size of the data

In [6]:
"""
Memo
----
1. medivo_test_result_type is a function of the following attributes: 
      "meta_sender_name",
      "receiving_organization_id",
      "test_order_code",
      "test_order_name",
      "test_result_code",
      "test_result_name",
      "test_result_loinc_code",
      "test_result_units_of_measure"
      
"""
from transformer import to_age
from analyzer import sample_col_values
from loinc import FeatureSet

cat_cols = FeatureSet.cat_cols

# ['patient_gender',
#  'patient_state',
#  'patient_bill_type',
#  'fasting',
#  'performing_organization_id',
#  'receiving_organization_id',
#  'test_result_status',
#  'test_order_code',
#  'test_order_name',
#  'test_result_code',
#  'test_result_name',
#  'test_result_value',
#  'test_result_range',
#  'test_result_abnormal_flag',
#  'test_result_reference_range',
#  'test_result_units_of_measure',
#  'test_result_comments',
#  'test_cpt_code',
#  'panel_order_code',
#  'panel_order_name',
#  'meta_sender_name',
#  'medivo_test_result_type']

cont_cols = FeatureSet.cont_cols  

target_cols = FeatureSet.target_cols # ['test_result_loinc_code', ]

derived_cols = FeatureSet.derived_cols
# ['count',  # due to resolve_duplicate()
# ]

# cardinality < 100
low_card_cols = ['patient_gender', 'fasting', 'meta_sender_name' ]
high_card_cols = list(set(cat_cols)-set(low_card_cols))

representative_cols = ["meta_sender_name",
      # "receiving_organization_id",
      # "test_order_code",
      "test_order_name",
      # "test_result_code",
      "test_result_name",
      "test_result_loinc_code",
      "test_result_units_of_measure"]

target_columns = cat_cols + cont_cols + target_cols

# feature transformation
#####################################################
# to_age(ts0)
# values = sample_col_values(ts0, col='age', n=10)
# print("> age: {}".format(values))

### Balance Classes 

note: balance classes from external dataset

In [7]:
from analyzer import balance_data_incr, load_data_incr, stratify
from transformer import resolve_duplicate
import loinc as lc
from loinc import LoincTSet

# run here or just load the curated data generated by analyzer.t_stratify()
tLoad = True
tSave = not tLoad

col_target = LoincTSet.col_target
token_default = token_missing = 'unknown'

ts = ts0
ds = stratify(ts, col='test_result_loinc_code', ascending=False)
print("(balance_classes) data size distribution:\n{}\n".format(ds[:25]))
ds = [(code, size) for code, size in ds if not code in lc.LoincTSet.non_codes]
max_size = ds[0][1] # ds[2][1]
codes_low_sz = set([code for code, size in ds if size < max_size])
print("(balance_classes) We have n={} with low sample size (< {})".format(len(codes_low_sz), max_size))

if tLoad: 
    input_file=f"andromeda-pond-{cohort}-balanced.csv"
    ts = load_src_data(input_file=input_file, warn_bad_lines=False)
    ts[col_target] = ts[col_target].astype(str)
    # ts = ts.drop_duplicates(keep='last')  # drop duplicates 
    # ts = lc.canonicalize(ts, col_target=col_target, token_missing=token_default, target_labels=loinc_set)
    if not lc.is_canonicalized(ts, col_target=col_target, token_missing=token_default, target_labels=loinc_set): 
        ts = ts.drop_duplicates(keep='last')  # drop duplicates 
        ts = lc.canonicalize(ts, col_target=col_target, token_missing=token_default) # noisy_values/[]
    
else: 
    # max_size = 1000
    input_file = f"andromeda-pond-{cohort}-loinc.csv"
    codes = set(loinc_set)
    codes_hit = set([])
    for i, tsi in enumerate(load_data_incr(input_file=input_file, chunksize=1000000, warn_bad_lines=False)): 
        N0 = ts.shape[0]
        # tsi = tsi.drop_duplicates(keep='last')  # drop duplicates 
        tsi = resolve_duplicate(tsi)
        tsi = lc.canonicalize(tsi, col_target=col_target, token_missing=token_default, target_labels=loinc_set)
        print("[{}] Processing chunk #{} | n(ts): {}, n(tsi): {} ...".format(i, i+1, N0, tsi.shape[0]))

        ts_incr, hit, missed = balance_data_incr(df=ts, df_extern=tsi, n_samples=max_size, col=col_target)
        if not ts_incr.empty: ts = pd.concat([ts, ts_incr])
            
        # --- analysis --- 
        N = ts.shape[0]
        codes_hit.union(hit) # still has this many codes without a match 

        print(f"[{i}] size: {N0} -> {N}")
        ds = stratify(ts, col='test_result_loinc_code', ascending=False)
        ds = [(code, size) for code, size in ds if size < max_size]
        print("[{}] size(codes) < {} (n={}): \n{}\n".format(i, max_size, len(ds), ds[:20]))

    codes_missed = codes_low_sz - codes_hit  
    print("(t_stratify) At last, we could not find a match for n={} codes among nl={} low-sample-size labels:\n{}\n".format(
        len(codes_missed), len(codes_low_sz), codes_missed))  
    
    # down sample control data (very often the negative examples are too huge)

    if tSave: 
        
        # should not have duplicates at this point (due to resolve_duplicate() call)
        # ts = ts.drop_duplicates(keep='last')  # drop duplicates 
        
        output_file = f"andromeda-pond-{cohort}-balanced.csv"
        save_data(ts, output_file=output_file)

"""
Output
------
   ts: balanced training data; well... as balaned as possible because external data may not be able to supply sufficient 
       data for a subset of the classes/loincs
"""

(balance_classes) data size distribution:
[('unknown', 12005), ('67686', 5703), ('17426', 4418), ('19752', 4408), ('17517', 4392), ('21600', 4167), ('7187', 3826), ('7773', 3783), ('178616', 3756), ('486431', 3735), ('30973', 3686), ('19208', 3589), ('66902', 3347), ('7310', 3045), ('28233', 3005), ('7112', 2791), ('7138', 2723), ('110114', 2559), ('7708', 2463), ('45443', 2413), ('7518', 2376), ('25718', 2241), ('134577', 2175), ('20933', 2165), ('20859', 2115)]

(balance_classes) We have n=1506 with low sample size (< 5703)
[load] Loading default input data: andromeda-pond-hepatitis-c-balanced.csv
(canonicalize) Operations: fill n/a + dehyphenate + replace_values + trim_tail + fill others (non-target classes)
(is_canonicalized) Found n=1 codes not in target set:
{'nan'}

(canonicalize) Operations: fill n/a + dehyphenate + replace_values + trim_tail + fill others (non-target classes)


'\nOutput\n------\n   ts: balanced training data; well... as balaned as possible because external data may not be able to supply sufficient \n       data for a subset of the classes/loincs\n'

### Feature Transformation

note: patient_date_of_birth => age

In [8]:
from transformer import to_age
from analyzer import col_values
# from loinc import FeatureSet

to_age(ts)
values = col_values(ts, col='age', n=10)
print("> age: {}".format(values))

# resolve_duplicate() call adds a new column: count (of duplicates)
# assert 'count' in ts.columns

# datatime columns

> age: [60 34 21 84 69 78 45 56 43 24]


### Subset Features and Handling missing values

In [9]:
tCategorify = False
tDropHighMissing = False # drop columns with high rate of missing values
p_null = 0.9
token_default = token_missing = 'unknown'

df = ts # ... :)

# V = list(feature_lookup.keys())
V = cont_cols + cat_cols # + derived_cols
L = target_cols
dfX = df[V]
dfy = df[L]

print("> Given features set:\n{}\n".format(V))

assert np.sum(dfy[target_cols[0]].isnull()) == 0

# drop columns/vars with too many missing values 
N = dfX.shape[0]
n_thresh = int(N * p_null)
nf0 = nf = dfX.shape[1]
fset0 = set(dfX.columns.values)

if tDropHighMissing: 
    dfX = dfX[dfX.columns[dfX.isnull().mean() < p_null]]
    fset = set(dfX.columns.values)
    nf = dfX.shape[1]
    print("> Dropped n={} features:\n{}\n".format(nf-nf0, fset0-fset))
    
fset = set(dfX.columns.values)
print("> Final feature set (nf={}):\n{}\n".format(nf, fset))

# fill in missing values (also see default_values)
dfX.fillna(value=token_default, inplace=True)
#################################################
# Convert our three categorical columns to category dtypes.

cat_cols = [cat for cat in cat_cols if cat in dfX.columns]
cont_cols = [c for c in cont_cols if c in dfX.columns]

"""
Output
------
   dfX 
   dfy
"""

> Given features set:
['age', 'patient_gender', 'patient_state', 'patient_bill_type', 'fasting', 'performing_organization_id', 'receiving_organization_id', 'test_result_status', 'test_order_code', 'test_order_name', 'test_result_code', 'test_result_name', 'test_result_value', 'test_result_range', 'test_result_abnormal_flag', 'test_result_reference_range', 'test_result_units_of_measure', 'test_result_comments', 'test_cpt_code', 'panel_order_code', 'panel_order_name', 'meta_sender_name', 'medivo_test_result_type']

> Final feature set (nf=23):
{'test_result_name', 'fasting', 'test_result_comments', 'test_result_units_of_measure', 'test_result_range', 'age', 'test_cpt_code', 'patient_state', 'test_result_status', 'test_result_reference_range', 'patient_bill_type', 'performing_organization_id', 'patient_gender', 'test_result_abnormal_flag', 'test_order_code', 'panel_order_code', 'receiving_organization_id', 'meta_sender_name', 'test_order_name', 'test_result_value', 'medivo_test_result_typ

'\nOutput\n------\n   dfX \n   dfy\n'

### Encode Variables

In [10]:
from transformer import encode_vars 
from loinc import FeatureSet
# high_card_cols = FeatureSet.high_card_cols
nf0 = dfX.shape[1]
dfX, encoder = encode_vars(dfX, fset=cat_cols, high_card_cols=high_card_cols)
print("> After variable encoding we have dim(dfX): {} | nf: {} -> {}".format(dfX.shape, nf0, dfX.shape[1]))
print("> New feature set:\n{}\n".format(dfX.columns))
print(df[representative_cols].head(10).to_string(index=False))

(encoder_vars2) low card vars (n=['meta_sender_name', 'patient_gender', 'fasting']):
3
 ... high card vars (n=['patient_state', 'test_result_name', 'test_result_status', 'test_result_reference_range', 'patient_bill_type', 'panel_order_name', 'performing_organization_id', 'test_result_comments', 'test_result_abnormal_flag', 'test_order_code', 'panel_order_code', 'receiving_organization_id', 'test_result_units_of_measure', 'test_order_name', 'test_result_range', 'test_result_value', 'medivo_test_result_type', 'test_cpt_code', 'test_result_code']):
19

... transforming var: patient_gender ...
... transforming var: patient_state ...
... transforming var: patient_bill_type ...
... transforming var: fasting ...
... transforming var: performing_organization_id ...
... transforming var: receiving_organization_id ...
... transforming var: test_result_status ...
... transforming var: test_order_code ...
... transforming var: test_order_name ...
... transforming var: test_result_code ...
... tran

### Encode Labels 

In [11]:
from analyzer import encode_labels, summarize_dict, get_sample_sizes
import collections, operator

# verify
assert dfX.shape[0] == dfy.shape[0], "> dim(dfX): {} | dfy.cols: {}".format(dfX.shape, dfy.columns.values)

codebook={'pos': 1, 'neg': 0, '+': 1, '-': 0}

# choose the one with a large sample size as 'positive'
col_label = 'test_result_loinc_code' # strings

topn = 5
sizes = get_sample_sizes(dfy[col_label])
# ... sizes: (loinc) label -> sample size
# print("> n(sizes): {}".format(len(sizes)))  # 734 for cohort='hepatitis-c'

# Q: How many classes/codes have less than N instances? 
N_low = 1000
n_low = sum(1 for l, c in sizes.items() if c < N_low)
print("> Low sample size classes | n={} (< {})".format(n_low, N_low))

N_elow = 10
n_elow = sum(1 for l, c in sizes.items() if c < N_elow)
print("> Extreme low sample size classes | n={} (< {})".format(n_elow, N_elow))

# Q: How many classes/codes were able to match the most enriched class in terms of sample size (e.g. 5707)? 
eps = 10
n_matched = sum(1 for l, c in sizes.items() if c >= max_size-eps)
print("> n_matched: {} | max_size: {}".format(n_matched, max_size))

# sort by values
sizes_sorted = sorted(sizes.items(), key=operator.itemgetter(1))
summarize_dict(sizes, topn=15, sort_=True)

print("> sizes: {}".format(sizes.most_common(20)))
most_sample_sizes = sizes.most_common(topn)  # take(topn, sizes.items())
print("> Sample sizes | Top N={} codes:\n{}\n".format(topn, most_sample_sizes))
least_sample_sizes = sizes.most_common()[:-topn-1:-1]
print("> Sample sizss | Last N={} codes:\n{}\n".format(topn, least_sample_sizes))

# test
target = most_sample_sizes[0][0]
y = encode_labels(dfy, pos_label=target, codebook=codebook, verbose=1)

> Low sample size classes | n=698 (< 1000)
> Extreme low sample size classes | n=395 (< 10)
> n_matched: 2 | max_size: 5703
[96107] -> 1
[772079] -> 1
[356188] -> 1
[160119] -> 1
[448134] -> 1
[159756] -> 1
[206649] -> 1
[80100] -> 1
[223123] -> 1
[71001] -> 1
[156679] -> 1
[530170] -> 1
[264663] -> 1
[52092] -> 1
[148049] -> 1
> sizes: [('nan', 980139), ('67686', 5872), ('17426', 4647), ('17517', 4602), ('19752', 4562), ('21600', 4379), ('unknown', 4267), ('178616', 4001), ('7773', 3938), ('7187', 3917), ('486431', 3861), ('30973', 3851), ('19208', 3726), ('66902', 3405), ('110114', 3253), ('28233', 3142), ('7310', 3104), ('7112', 2848), ('7138', 2827), ('7708', 2503)]
> Sample sizes | Top N=5 codes:
[('nan', 980139), ('67686', 5872), ('17426', 4647), ('17517', 4602), ('19752', 4562)]

> Sample sizss | Last N=5 codes:
[('76919', 1), ('206060', 1), ('62372', 1), ('162339', 1), ('64204', 1)]

(encode_labels) sample size: Counter({1: 980139, 0: 143360})


## Initial Model Training

### 1. Feature Selection

In [12]:
"""
Ref
---
1. pip install feature-selector

   https://github.com/WillKoehrsen/feature-selector
   
   possible dependency 
      brew install libomp
      
   <debug> 
       + RuntimeError: Python is not installed as a framework.
          > https://stackoverflow.com/questions/34977388/matplotlib-runtimeerror-python-is-not-installed-as-a-framework
   
"""
import feature_selector 

### 2. Save a copy of encoded training set (optional)

In [13]:
tSaveEncodedTSet = True 

if tSaveEncodedTSet: 
    output_file = f"andromeda-pond-{cohort}-encoded.csv"
    save_data(ts, output_file=output_file)
    

(save_data) Saved dataframe (dim=(1123499, 129)) to:
/Users/barnett/Documents/work/loinc_predictor/data/andromeda-pond-hepatitis-c-encoded.csv



### 3. Model Training

In [14]:
import utils_tree, utils_sys, analyzer
import collections
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from analyzer import balance_by_downsampling

# data transformation
col = 'test_result_loinc_code'
X, y = dfX.values, dfy[col].values
print("> dim(X): {}, sample(y): {}".format(X.shape, np.random.choice(np.unique(y),20) ))

# feature scaling
scaler = MinMaxScaler() # MinMaxScaler(), StandardScaler()
X = scaler.fit_transform(X)

n_fold = 5
n_min = n_fold

# to save performance data
header = ['code', 'mean', 'std', 'n_pos']
sdict = {h:[] for h in header}
for code in loinc_set: 
    y_eff = analyzer.encode_labels(y, pos_label=code)
    
    counter = collections.Counter(y_eff)
    n_pos, n_neg = counter[codebook['pos']], counter[codebook['neg']]
    print("> sample size | n(+): {}, n(-): {} | dim(y_encoded): {}".format(n_pos, n_neg, y_eff.shape))
    
    if n_pos >= n_min: 
        # downsampling majority classes (very often we have too many negative instances)
        X_eff, y_eff = balance_by_downsampling(X, y_eff, method='multiple', majority_max=3)
        n_pos, n_neg = counter[codebook['pos']], counter[codebook['neg']]
        print(f"> sample size (after downsampling majority) | n(+): {n_pos}, n(-): {n_neg}")
        
        scores = analyzer.eval_performance(X_eff, y_eff, model=None, cv=n_fold, random_state=53, verbose=1)
        mean_score = np.mean(scores)
        std_score = np.std(scores)
        print("> average: {}, std: {}".format(mean_score, std_score))
    else: 
        print("> (positive) sample size too small, n={}".format(n_pos))
        mean_score = -1 
        std_score = -1
    sdict['code'].append(code)
    sdict['mean'].append(mean_score)
    sdict['std'].append(std_score)
    sdict['n_pos'].append(n_pos)

# --------------------------------------------------
# save performance dataframe
df_perf = DataFrame(sdict, columns=header)
df_perf = df_perf.sort_values(by=['mean', ]) # ascending=False
analyzer.save_performance(df, output_dir='result') # output_file/'' (performance-<cohort>.csv)

cohort = 'hepatitis-c'
output_dir = os.path.join(os.getcwd(), 'result')
output_file = f"performance-{cohort}-0.csv" 
output_path = os.path.join(output_dir, output_file)
df_perf.to_csv(output_path, sep='|', index=False, header=True)

for code, score in zip(df_perf['code'], df_perf['mean']):
    print(f"[{code}] -> {score}")

> dim(X): (1123499, 243), sample(y): ['56853' '7708' '103317' '295980' '25007' '339440' '115725' '66902'
 '169151' '10058' '39347' '160119' '177915' '21600' '505610' '28886'
 '204545' '7096' '143149' '139907']
(encode_labels) sample size: Counter({0: 1118852, 1: 4647})
> sample size | n(+): 4647, n(-): 1118852 | dim(y_encoded): (1123499,)
(balance_by_downsampling) dim(X): (1123499, 243), nl: 2, labels: [0 1] | nf=243
... After merging (X, y) => dim(X): (1123499, 244)
> sample size (after downsampling majority) | n(+): 4647, n(-): 1118852
> 0 of KFold 5
> Fmax: 0.9983879634605051 p_th: 0.92472648938096 | F1: 0.9973161567364466, AUC: 0.9999791584731806
> 1 of KFold 5
> Fmax: 0.998389694041868 p_th: 0.6156882545750916 | F1: 0.998389694041868, AUC: 0.9999911294179356
> 2 of KFold 5
> Fmax: 0.9978517722878624 p_th: 0.9223685692664245 | F1: 0.9967845659163987, AUC: 0.999987658320606
> 3 of KFold 5
> Fmax: 1.0 p_th: 0.9115691417877637 | F1: 0.9989247311827958, AUC: 1.0
> 4 of KFold 5
> Fmax: 

> sample size | n(+): 973, n(-): 1122526 | dim(y_encoded): (1123499,)
(balance_by_downsampling) dim(X): (1123499, 243), nl: 2, labels: [0 1] | nf=243
... After merging (X, y) => dim(X): (1123499, 244)
> sample size (after downsampling majority) | n(+): 973, n(-): 1122526
> 0 of KFold 5
> Fmax: 0.9974293059125964 p_th: 0.7930443192916324 | F1: 0.9948717948717949, AUC: 0.9998682824025289
> 1 of KFold 5
> Fmax: 1.0 p_th: 0.841096526684512 | F1: 0.9974424552429668, AUC: 1.0
> 2 of KFold 5
> Fmax: 0.9948717948717948 p_th: 0.8630448451505258 | F1: 0.9948717948717948, AUC: 0.9999735206891682
> 3 of KFold 5
> Fmax: 1.0 p_th: 0.7245684662283568 | F1: 1.0, AUC: 1.0
> 4 of KFold 5
> Fmax: 1.0 p_th: 0.956380427368394 | F1: 0.9873417721518987, AUC: 1.0
> average: 0.9984602201568784, std: 0.0020519427916753322
(encode_labels) sample size: Counter({0: 1119120, 1: 4379})
> sample size | n(+): 4379, n(-): 1119120 | dim(y_encoded): (1123499,)
(balance_by_downsampling) dim(X): (1123499, 243), nl: 2, labe

> Fmax: 0.9991235758106924 p_th: 0.7138506700874838 | F1: 0.9965034965034965, AUC: 0.999998972841941
> average: 0.9994735288291142, std: 0.00042986338535625304
(encode_labels) sample size: Counter({0: 1119582, 1: 3917})
> sample size | n(+): 3917, n(-): 1119582 | dim(y_encoded): (1123499,)
(balance_by_downsampling) dim(X): (1123499, 243), nl: 2, labels: [0 1] | nf=243
... After merging (X, y) => dim(X): (1123499, 244)
> sample size (after downsampling majority) | n(+): 3917, n(-): 1119582
> 0 of KFold 5
> Fmax: 0.9993618379068284 p_th: 0.6684909275431046 | F1: 0.9993618379068284, AUC: 0.9999994567676699
> 1 of KFold 5
> Fmax: 0.9993618379068284 p_th: 0.9525800480928899 | F1: 0.996817313812858, AUC: 0.999997286148502
> 2 of KFold 5
> Fmax: 1.0 p_th: 0.9594603514607686 | F1: 0.9974554707379135, AUC: 1.0
> 3 of KFold 5
> Fmax: 0.9993618379068284 p_th: 0.916647674724642 | F1: 0.9974522292993631, AUC: 0.9999994565365072
> 4 of KFold 5
> Fmax: 0.9974457215836526 p_th: 0.8307176172145435 | F1

> Fmax: 0.995405819295559 p_th: 0.9094634473634876 | F1: 0.9938837920489296, AUC: 0.9993608133669609
> 4 of KFold 5
> Fmax: 0.995398773006135 p_th: 0.8309598426667172 | F1: 0.9946360153256705, AUC: 0.9993913869841847
> average: 0.9955576613564092, std: 0.0012222869210699707
(encode_labels) sample size: Counter({0: 1122425, 1: 1074})
> sample size | n(+): 1074, n(-): 1122425 | dim(y_encoded): (1123499,)
(balance_by_downsampling) dim(X): (1123499, 243), nl: 2, labels: [0 1] | nf=243
... After merging (X, y) => dim(X): (1123499, 244)
> sample size (after downsampling majority) | n(+): 1074, n(-): 1122425
> 0 of KFold 5
> Fmax: 0.9907407407407408 p_th: 0.8469155127278413 | F1: 0.9884526558891455, AUC: 0.9990769785469623
> 1 of KFold 5
> Fmax: 0.9907407407407407 p_th: 0.5032524542905642 | F1: 0.9907407407407407, AUC: 0.9986452220531767
> 2 of KFold 5
> Fmax: 0.9976798143851509 p_th: 0.8647429553903304 | F1: 0.9953703703703703, AUC: 0.9999205546728297
> 3 of KFold 5
> Fmax: 0.995370370370370

> Fmax: 1.0 p_th: 0.922054958210193 | F1: 0.9989888776541962, AUC: 1.0
> average: 0.999392915162501, std: 0.0004956849434250422
(encode_labels) sample size: Counter({0: 1123485, 1: 14})
> sample size | n(+): 14, n(-): 1123485 | dim(y_encoded): (1123499,)
(balance_by_downsampling) dim(X): (1123499, 243), nl: 2, labels: [0 1] | nf=243
... After merging (X, y) => dim(X): (1123499, 244)
> sample size (after downsampling majority) | n(+): 14, n(-): 1123485
> 0 of KFold 5
> Fmax: 1.0 p_th: 0.9696668131916392 | F1: 1.0, AUC: 1.0
> 1 of KFold 5
> Fmax: 1.0 p_th: 0.9724800618723248 | F1: 1.0, AUC: 1.0
> 2 of KFold 5
> Fmax: 1.0 p_th: 0.9369812336284101 | F1: 1.0, AUC: 1.0
> 3 of KFold 5
> Fmax: 1.0 p_th: 0.9371025361818622 | F1: 1.0, AUC: 1.0
> 4 of KFold 5
> Fmax: 1.0 p_th: 0.1962643615339869 | F1: 0.8, AUC: 1.0
> average: 1.0, std: 0.0
(encode_labels) sample size: Counter({0: 1121047, 1: 2452})
> sample size | n(+): 2452, n(-): 1121047 | dim(y_encoded): (1123499,)
(balance_by_downsampling) di

...

> (positive) sample size too small, n=1
(encode_labels) sample size: Counter({0: 1123496, 1: 3})
> sample size | n(+): 3, n(-): 1123496 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=3
(encode_labels) sample size: Counter({0: 1123498, 1: 1})
> sample size | n(+): 1, n(-): 1123498 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=1
(encode_labels) sample size: Counter({0: 1123498, 1: 1})
> sample size | n(+): 1, n(-): 1123498 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=1
(encode_labels) sample size: Counter({0: 1123497, 1: 2})
> sample size | n(+): 2, n(-): 1123497 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=2
(encode_labels) sample size: Counter({0: 1123496, 1: 3})
> sample size | n(+): 3, n(-): 1123496 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=3
(encode_labels) sample size: Counter({0: 1123492, 1: 7})
> sample size | n(+): 7, n(-): 1123492 | dim(y_encoded): (1123499,)
(balance_b

(encode_labels) sample size: Counter({0: 1123495, 1: 4})
> sample size | n(+): 4, n(-): 1123495 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=4
(encode_labels) sample size: Counter({0: 1123498, 1: 1})
> sample size | n(+): 1, n(-): 1123498 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=1
(encode_labels) sample size: Counter({0: 1123487, 1: 12})
> sample size | n(+): 12, n(-): 1123487 | dim(y_encoded): (1123499,)
(balance_by_downsampling) dim(X): (1123499, 243), nl: 2, labels: [0 1] | nf=243
... After merging (X, y) => dim(X): (1123499, 244)
> sample size (after downsampling majority) | n(+): 12, n(-): 1123487
> 0 of KFold 5
> Fmax: 1.0 p_th: 0.9729316785092583 | F1: 1.0, AUC: 1.0
> 1 of KFold 5
> Fmax: 1.0 p_th: 0.8951380063927971 | F1: 1.0, AUC: 1.0
> 2 of KFold 5
> Fmax: 1.0 p_th: 0.8320158914887106 | F1: 1.0, AUC: 1.0
> 3 of KFold 5
> Fmax: 1.0 p_th: 0.8774068446763696 | F1: 1.0, AUC: 1.0
> 4 of KFold 5
> Fmax: 1.0 p_th: 0.8262875770506514 | 

(encode_labels) sample size: Counter({0: 1123497, 1: 2})
> sample size | n(+): 2, n(-): 1123497 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=2
(encode_labels) sample size: Counter({0: 1123495, 1: 4})
> sample size | n(+): 4, n(-): 1123495 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=4
(encode_labels) sample size: Counter({0: 1123498, 1: 1})
> sample size | n(+): 1, n(-): 1123498 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=1
(encode_labels) sample size: Counter({0: 1123492, 1: 7})
> sample size | n(+): 7, n(-): 1123492 | dim(y_encoded): (1123499,)
(balance_by_downsampling) dim(X): (1123499, 243), nl: 2, labels: [0 1] | nf=243
... After merging (X, y) => dim(X): (1123499, 244)
> sample size (after downsampling majority) | n(+): 7, n(-): 1123492
> 0 of KFold 5
> Fmax: 1.0 p_th: 0.8600280846302669 | F1: 1.0, AUC: 1.0
> 1 of KFold 5
> Fmax: 1.0 p_th: 0.6216474987248386 | F1: 1.0, AUC: 1.0
> 2 of KFold 5
> Fmax: 1.0 p_th: 0.9

> sample size | n(+): 1, n(-): 1123498 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=1
(encode_labels) sample size: Counter({0: 1123497, 1: 2})
> sample size | n(+): 2, n(-): 1123497 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=2
(encode_labels) sample size: Counter({0: 1123498, 1: 1})
> sample size | n(+): 1, n(-): 1123498 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=1
(encode_labels) sample size: Counter({0: 1123498, 1: 1})
> sample size | n(+): 1, n(-): 1123498 | dim(y_encoded): (1123499,)
> (positive) sample size too small, n=1
(save) Saving performance dataframe to:
/Users/barnett/Documents/work/loinc_predictor/result/performance-hepatitis-c.csv
 ... #


KeyError: 'code'

### Visualize Results

In [ ]:
"""

Memo
---- 
1. performance plot

   perplot: https://pypi.org/project/perfplot/
"""
import seaborn as sns
import matplotlib.pyplot as plt
from analyzer import load_performance

sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(6, 20))
sns.set_color_codes("pastel")

#---------------------------------------------

# load performance data
cohort = 'hepatitis-c'
df_perf = load_performance(input_dir='result', cohort=cohort)
print("> dim(performance matrix): {}".format(df_perf.shape))

# sort ~ performance scores 
# df_perf = df_perf.sort_values(by=['mean', ], ascending=False)

header = ['code', 'mean', 'std', 'n_pos']
codes = df_perf['code']
n_codes = len(codes)
scores = df_perf['mean']

# some statistics
score_high = 0.90
score_low = 0.50

codes_low_sz = df_perf.loc[df_perf['mean'] < 0]['code']
codes_scored = df_perf.loc[df_perf['mean'] >= 0]['code']
codes_high_score = df_perf.loc[df_perf['mean'] >= score_high]['code']
assert n_codes == len(codes_low_sz) + len(codes_scored)

print("1. Total number of codes: {} | n(low_sample): {}, n(scored):{}, n(high scored):{}".format(n_codes, 
   len(codes_low_sz), len(codes_scored), len(codes_high_score)))
r_scored = len(codes_scored)/(n_codes+0.0)
rh = len(codes_high_score)/(n_codes+0.0)
print("2. Fraction of scored codes: {}".format(r_scored))
print("3. Fraction of highly scored codes: {}".format(rh))

# Effective performance dataframe, ruling out those codes without scores (due to low sample sizes)
df_eff = df_perf.loc[df_perf['mean'] >= 0.0]

n_offset = 25
df_topn = df_eff.sort_values(['mean', ], ascending=False).head(n_offset)
df_botn = df_eff.sort_values(['mean', ], ascending=True).head(n_offset)
# print(df_botn)

# codes = [str(c) for c in df_botn['code'].values]
# print('lower codes: {}'.format(codes))
# scores = df_botn['mean'].values
# print('scores: {}'.format(scores))

# top n + bottom n
dfe = pd.concat([df_topn, df_botn], ignore_index=True)
dfe.sort_values(by=['mean', ], ascending=False, inplace=True)
codes = [str(c) for c in dfe['code'].values]
scores = dfe['mean'].values
# print('lower(n)+higher codes(n): {}'.format(codes))
# print('scores: {}'.format(scores))
print(dfe)

# sns.barplot(x="total", y="abbrev", data=crashes,
#             label="Total", color="b")

# --------------------
# ax = sns.barplot(x='mean', y='code', data=df_botn)
# print("-------------------------\n\n")
# print("> dtype: {}".format(df_botn.dtypes))
# print(df_botn.head(10))

# dfe = dfe[['mean', 'code']]
# dfe.plot(kind='bar')

sns.barplot(x='mean', y='code', data=dfe, order=dfe['code'], # order has to be specified; even if already sorted!!!
            label="LOINC", color="b", orient='h')

# ax = sns.barplot(x='mean', y='code', data=df)

# ax.set_xlabel('Fmax Score')
# ax.set_ylabel('LOINC')
